In [ ]:
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.metrics import root_mean_squared_error
import plotly.express as px
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
bpac3 = yf.Ticker("BPAC3.SA")
df_bpac3 = bpac3.history(start="2021-01-01",end="2023-12-30")

total_rows = len(df_bpac3)
legs = 21

df_train = df_bpac3.iloc[:total_rows - legs]
df_test = df_bpac3.iloc[total_rows - legs:]

training_set_feature = df_train[['High', 'Low', 'Close']].values


In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps, X_in, y_in):
	X, y = X_in, y_in
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return X, y

In [ ]:
X, y = list(), list()
qtd_features = 2
qtd_goal = 1
for index in range(len(training_set_feature)):
    X, y = split_sequence(training_set_feature[index], qtd_features, X, y)
X, y = np.array(X), np.array(y)


rnn = Sequential()

rnn.add(LSTM(units=50, return_sequences=True, input_shape=(qtd_features, qtd_goal)))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50))
rnn.add(Dropout(0.5))

rnn.add(Dense(units=1))
rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
'''
Epochs -> Quantidade de vezes que o modelo vai treinar
'''
rnn.fit(X, y, epochs = 200, batch_size = 32)
x_input = np.array([17.746909, 17.568945])
x_input = x_input.reshape((qtd_goal, qtd_features))
yhat = rnn.predict(x_input, verbose=0)
yhat

In [ ]:


df = pd.DataFrame(relacao)

fig = px.line(
    df, 
    x='epochs',
    y=['rmse'],
    
)
fig.update_layout(title="Relação Épocas x RMSE obtido para uma RNN de janela 3 com 5 camadas para ação BPAC3", xaxis_title="Épocas", yaxis_title="RMSE Obtido") 

fig.show()